<a href="https://colab.research.google.com/github/Laurahg22/Trabajo-analitica-RH/blob/main/e_despliegue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Conectar el drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd ### para manejo de datos
import sqlite3 as sql
import joblib
import openpyxl ## para exportar a excel
import numpy as np
import sys #para ver la ruta
from sklearn.preprocessing import LabelEncoder

sys.path ###Ruta directorio qué tiene paquetes
sys.path.append('/content/drive/MyDrive/trabajo/Trabajo-analitica-RH') ## este comanda agrega una ruta

import a_funciones as funciones  ###archivo de funciones propias

In [3]:
### funcion para cargar objeto guardado ###
df_2016_d =joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/df_dummies_2016.pkl")

#Automatizar

In [4]:
if __name__=="__main__":
    ## Cargar base de datos
    df_2016_d
    #### Otras transformaciones en python (imputación, dummies y selección de variables)
    df_t = funciones.preparar_datos(df_2016_d)

    ## Cargar modelo y predecir
    dt_final = joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/dt_final.pkl")
    predicciones = dt_final.predict(df_t)
    pd_pred = pd.DataFrame(predicciones, columns=['Retiros'])

    ### Crear base con predicciones ###
    perf_pred = pd.concat([df_2016_d['EmployeeID'], df_t, pd_pred], axis=1)

    ####LLevar a BD para despliegue
    perf_pred[['EmployeeID', 'Retiros']].to_csv("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/perf_pred.csv", index=False)

    #### Calcular la tasa de retiros ###
    total_empleados = len(perf_pred)
    retirados = perf_pred['Retiros'].sum()
    tasa_retiros = (retirados / total_empleados) * 100

    #### Crear DataFrame con la tasa de retiros y agregarlo a la información de empleados que se van a retirar ###
    tasa_retiros = pd.DataFrame({"Tasa de Retiros": [tasa_retiros]})
    emp_que_renuncian = perf_pred.loc[perf_pred["Retiros"] == 1]
    emp_que_renuncian_with_tasa = pd.concat([emp_que_renuncian, tasa_retiros], axis=1)
    coeficientes = pd.DataFrame(np.append([np.nan], [np.nan]), columns=['coeficientes'])  ### agregar coeficientes

    #### Exportar la información de empleados que se van a retirar, incluyendo la tasa de retiros, a un archivo Excel ###
    emp_que_renuncian_with_tasa.to_excel("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/empleados_que_se_retiran.xlsx", index=False)
    coeficientes.to_excel("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/coeficientes.xlsx") ### exportar coeficientes para analizar predicciones